In [1]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 31.9 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login

login()

In [5]:
import numpy as np
import pandas as pd
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [6]:
model_name = "meta-llama/Llama-2-13b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model in 4-bit precision
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype="auto"
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Add data

In [7]:
file_path = r"/content/all_data.csv"
data = pd.read_csv(file_path)

<ipython-input-7-08146613f25a>:2: DtypeWarning: Columns (8,9,31,32,36) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [8]:
tasks = ['Free recall', 'Lexical decision', 'Cued recall', 'Single recognition', 'Associative recognition']

In [9]:
task_data = {task: data[data['condition'] == task] for task in tasks}

In [10]:
cleaned_tasks = {}

cleaned_tasks['Free recall'] = task_data['Free recall'][['subject', 'trial', 'resp.string','stim.string.left']].dropna()
cleaned_tasks['Lexical decision'] = task_data['Lexical decision'][['subject', 'stim.string.left', 'resp', 'rt']].dropna()
cleaned_tasks['Cued recall'] = task_data['Cued recall'][['subject', 'trial', 'stim.string.left', 'resp.string']].dropna()
cleaned_tasks['Single recognition'] = task_data['Single recognition'][['subject', 'stim.string.left', 'resp']].dropna()
cleaned_tasks['Associative recognition'] = task_data['Associative recognition'][['subject', 'stim.string.left', 'stim.string.right', 'resp']].dropna()

- Cued Recall

In [11]:
cued_recall_data = cleaned_tasks['Cued recall']

In [12]:
cued_recall_data.shape

(10872, 4)

In [13]:
cued_recall_data.head()

,subject,trial,stim.string.left,resp.string
35,2,3,SERIOUSLY,tough
36,2,4,PARTIES,bedroom
38,2,6,REACHES,continue
40,2,8,SAVED,location
41,2,9,CONNECTED,continue


In [60]:
def generate_prompt_for_cued_recall(cue):
    return (
        "You are an expert at remembering associations between words. "
        f"Recall the word associated with the following cue:\nCue: \"{cue}\""
        "\nProvide your response as a single word only."
    )

In [18]:
sample_cued_recall_data = cued_recall_data.sample(5)
sample_cued_recall_data

,subject,trial,stim.string.left,resp.string
65034,297,12,HUNT,rarely
47139,212,5,ARRANGEMENT,seeds
114881,374,5,BOATS,system
9866,47,1,WORN,q
96409,435,4,MAIL,efficient


In [16]:
def ask_llm_v2(prompt, tokenizer, model):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.2,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id  # Force model to stop at eos_token
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [17]:
def get_model_responses_cued_recall(data, tokenizer, model):
    model_responses = []

    for _, row in data.iterrows():
        cue = row['stim.string.left']

        prompt = generate_prompt_for_cued_recall(cue)
        print(f"--Prompt--\n{prompt}\n")

        response = ask_llm_v2(prompt, tokenizer=tokenizer, model=model)
        print(f"--Model Raw Response--\n{response}\n")

        cleaned_response = response.strip().split()[-1]
        print(f"--Cleaned Model Response--\n{cleaned_response}\n")

        model_responses.append(cleaned_response)

    return model_responses

In [19]:
model_responses = get_model_responses_cued_recall(sample_cued_recall_data, tokenizer, model)
print("Sample Model Responses:", model_responses)

--Prompt--
You are an expert at remembering associations between words. Recall the word associated with the following cue:
Cue: "HUNT"
Provide your response as a single word only.



/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


--Model Raw Response--
You are an expert at remembering associations between words. Recall the word associated with the following cue:
Cue: "HUNT"
Provide your response as a single word only.

--Cleaned Model Response--
only.

--Prompt--
You are an expert at remembering associations between words. Recall the word associated with the following cue:
Cue: "ARRANGEMENT"
Provide your response as a single word only.

--Model Raw Response--
You are an expert at remembering associations between words. Recall the word associated with the following cue:
Cue: "ARRANGEMENT"
Provide your response as a single word only.

--Cleaned Model Response--
only.

--Prompt--
You are an expert at remembering associations between words. Recall the word associated with the following cue:
Cue: "BOATS"
Provide your response as a single word only.

--Model Raw Response--
You are an expert at remembering associations between words. Recall the word associated with the following cue:
Cue: "BOATS"
Provide your response

I am getting not good results

Then I THOUGHT, I am not proving the model first inputs to let them understand these pairs.
if we are generating 10 samples to test, we should give model these pair examples to model to understand them, then give them one cue from the sample now ask them to provide the next word, then it will be fair comparison between human and model. Otherwise, it is biased towards human, because humans knew these pairs beforehand in study phase and after that they were asked to remember!

In [20]:
def generate_study_prompt(pairs):
    prompt = "Study the following cue-response pairs:\n"
    for cue, response in pairs:
        prompt += f"{cue} - {response}\n"
    return prompt

In [53]:
def generate_test_prompt(cue):
    return (
        f"Recall the word associated with the following cue:\n{cue}.\n"
        f"Think carefully based on the pattern you recognized during the study phase. "
        f"Provide your answer with the starting format - \"{cue}\": "
    )


In [56]:
def generate_test_prompt_V2(cue):
    return (
        f"Recall the word associated with the following cue:\n{cue}.\n"
        f"Answer your response regarding the pattern you recognized from the study phase. "
        f"Keep in mind that you will be given a new test data, so do not try to find the answer to your current cue from the study phase. "
        f"Instead, try to understand the pattern and intuition between the words in the study phase. "
        f"Provide your answer starting with - \"{cue}\": "
    )


In [37]:
def get_model_responses_cued_recall(data, tokenizer, model):

    # Step 1: Split into study and test sets
    data = data.sample(frac=1).reset_index(drop=True)
    study_size = int(len(data) * 0.6)  # 60% for study, 40% for test
    study_data = data.iloc[:study_size]
    test_data = data.iloc[study_size:]

    study_pairs = list(zip(study_data['stim.string.left'], study_data['resp.string']))
    study_prompt = generate_study_prompt(study_pairs)


    model_responses = []
    for cue in test_data['stim.string.left']:
        test_prompt = study_prompt + "\n" + generate_test_prompt(cue)
        print("\n---------------")
        print(f"Test prompt: {test_prompt}")
        print("\n---------------")
        response = ask_llm_v2(test_prompt, tokenizer, model)
        print(f"\nResponse\n: {response}")
        model_response = input("Manually TYPE model output: ")
        print("\n---------------")
        model_responses.append(model_response)

    test_data = test_data.copy()
    test_data['model_response'] = model_responses
    return test_data

In [38]:
cued_recall_data_sample = cued_recall_data.sample(10)

In [39]:
test_results = get_model_responses_cued_recall(cued_recall_data_sample, tokenizer, model)


---------------
Test prompt: Study the following cue-response pairs:
TOWARDS - politics
DISAPPEARED - tounge
RAPID - tribes
ROPE - soft
COUPLE - characters
FENCE - situations

Recall the word associated with the following cue:
CIRCUMSTANCES: 

---------------

Response
: Study the following cue-response pairs:
TOWARDS - politics
DISAPPEARED - tounge
RAPID - tribes
ROPE - soft
COUPLE - characters
FENCE - situations

Recall the word associated with the following cue:
CIRCUMSTANCES: 

What is the word associated with the cue CIRCUMSTANCES?

Answer: SITUATIONS.
Manually TYPE model output: SITUATIONS

---------------

---------------
Test prompt: Study the following cue-response pairs:
TOWARDS - politics
DISAPPEARED - tounge
RAPID - tribes
ROPE - soft
COUPLE - characters
FENCE - situations

Recall the word associated with the following cue:
TRANSPORTATION: 

---------------

Response
: Study the following cue-response pairs:
TOWARDS - politics
DISAPPEARED - tounge
RAPID - tribes
ROPE - sof

In [40]:
test_results

,subject,trial,stim.string.left,resp.string,model_response
6,163,1,CIRCUMSTANCES,Load,SITUATIONS
7,433,4,TRANSPORTATION,wings,RAPID
8,127,12,EXPANSION,sad,RAPID
9,136,14,HIGHEST,structures,TOWARDS


In [48]:
def ask_llm_v3(prompt, tokenizer, model):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
        **inputs,
        max_new_tokens=70,
        temperature=0.7, # I reduced temp
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [61]:
def get_model_responses_cued_recall_V2(data, tokenizer, model):

    # Step 1: Split into study and test sets
    data = data.sample(frac=1).reset_index(drop=True)
    study_size = int(len(data) * 0.75)  # 75% for study, 25% for test
    study_data = data.iloc[:study_size]
    test_data = data.iloc[study_size:]

    study_pairs = list(zip(study_data['stim.string.left'], study_data['resp.string']))
    study_prompt = generate_study_prompt(study_pairs)


    model_responses = []
    cues= []
    for cue in test_data['stim.string.left']:
        cues.append(cue)
        test_prompt = study_prompt + "\n" + generate_test_prompt(cue)
        print("\n---------------")
        print(f"Test prompt: {test_prompt}")
        print("\n---------------")
        response = ask_llm_v3(test_prompt, tokenizer, model)
        print(f"\nResponse\n: {response}")
        model_response = input("Manually TYPE model output: ")
        print("\n---------------")
        model_responses.append(model_response)

    test_data = test_data.copy()
    test_data['cues'] = cues
    test_data['model_response'] = model_responses

    return test_data

In [62]:
cued_recall_data_sample = cued_recall_data.sample(20)

In [64]:
test_results2 = get_model_responses_cued_recall_V2(cued_recall_data_sample, tokenizer, model)


---------------
Test prompt: Study the following cue-response pairs:
NURSE - advice
SHEET - firms
ROOF - beings
EGG - roman
GATE - rode
STANDARDS - equivlent
BURN - steam
PRIDE - rode
POCKET - start
SCATTERED - practical
PAPERS - trail
NEGATIVE - acuracy
RESISTANCE - marriage
KINGDOM - vegetables
VEGETABLES - quantity

Recall the word associated with the following cue:
TRADITIONAL.
Think carefully based on the pattern you recognized during the study phase. Provide your answer with the starting format - "TRADITIONAL": 

---------------

Response
: Study the following cue-response pairs:
NURSE - advice
SHEET - firms
ROOF - beings
EGG - roman
GATE - rode
STANDARDS - equivlent
BURN - steam
PRIDE - rode
POCKET - start
SCATTERED - practical
PAPERS - trail
NEGATIVE - acuracy
RESISTANCE - marriage
KINGDOM - vegetables
VEGETABLES - quantity

Recall the word associated with the following cue:
TRADITIONAL.
Think carefully based on the pattern you recognized during the study phase. Provide your a

In [65]:
test_results2

,subject,trial,stim.string.left,resp.string,cues,model_response
15,343,6,TRADITIONAL,row,TRADITIONAL,ROOF
16,312,13,BEAT,1,BEAT,RESISTANCE
17,33,5,BLEW,gas,BLEW,BLUE
18,297,18,FILE,household,FILE,None
19,307,14,ROUTE,driver,ROUTE,Trail


In [67]:
test_results2["match"] = test_results2["resp.string"] == test_results2["model_response"]

In [68]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt

In [69]:
accuracy = test_results2["match"].mean()
precision = precision_score(test_results2["resp.string"], test_results2["model_response"], average="micro")
recall = recall_score(test_results2["resp.string"], test_results2["model_response"], average="micro")
f1 = f1_score(test_results2["resp.string"], test_results2["model_response"], average="micro")


In [70]:
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Accuracy: 0.00
Precision: 0.00
Recall: 0.00
F1-Score: 0.00


### Understandable results!
- I have tested this prompt on very few sample
- My model is not capable of giving out outputs in appropriate format bein an untuned model.
- Echoing the input prompt is an consistent problem
- MAINLY, thee model as well as the prompt lacks CONSISTENCY, sometimes it generates pretty gud results sometimes not!
